# Setup Environment

## Install KVM on Kernel 5.4.24-1-MANJARO x86_64
https://www.fosslinux.com/2484/how-to-install-virtual-machine-manager-kvm-in-manjaro-and-arch-linux.htm

https://computingforgeeks.com/complete-installation-of-kvmqemu-and-virt-manager-on-arch-linux-and-manjaro/

## Download Ubuntu ISO Bionic
http://releases.ubuntu.com/18.04.4/ubuntu-18.04.4-desktop-amd64.iso

## QEMU
Networking arch: https://wiki.qemu.org/Documentation/Networking

### Create a Disk Image
```
sudo qemu-img create -f qcow2 /var/lib/libvirt/images/pool/openair-cn 10G
```

### Launch QEMU with Ubuntu ISO attached
Run the following command to launch a QEMU virtual machine with the Ubuntu ISO and hard disk we created in the previous step attached.

```
sudo qemu-system-x86_64 \
  -m 4096 \
  -smp 2 \
  -vga virtio \
  -show-cursor \
  -usb \
  -device usb-tablet \
  -enable-kvm \
  -cdrom ubuntu-18.04.4-desktop-amd64.iso \
  -drive file=/var/lib/libvirt/images/pool/openair-cn,format=qcow2,if=virtio \
  -cpu host

```
The iso must be located in the same place where the command is executed.

Use _-monitor stdio_ to open monitor.

### Reboot without Ubuntu ISO attached
Once the machine is powered off. Run the command below to power it back on without the CD-ROM attached.

```
sudo qemu-system-x86_64 \
  -m 4096 \
  -smp 2 \
  -vga virtio \
  -show-cursor \
  -usb \
  -device usb-tablet \
  -enable-kvm \
  -drive file=/var/lib/libvirt/images/pool/openair-cn,format=qcow2,if=virtio \
  -cpu host \
  -net nic,model=virtio \
  -net user,hostfwd=tcp::2222-:22
```

Ref: https://graspingtech.com/ubuntu-desktop-18.04-virtual-machine-macos-qemu/

TODO: Start ubuntu in console mode
https://askubuntu.com/questions/859630/how-to-start-ubuntu-in-console-mode

## Install Packages in Guest 
In guest, open terminal.

```
sudo apt update
sudo apt upgrade -y
sudo apt install openssh-server git curl vim tmux
```

## Access Guest from Host
Open terminal and you copy public key and then loggin on guest.

```
ssh-copy-id openair-cn@localhost -p 2222
ssh openair-cn@localhost -p 2222
```

# Open Air Interface Core Network ( OAI-CN )
Cool tool:
https://www.gitmemory.com/openairinterface-GH

Tutorial Cassandra: 
https://www.tutorialspoint.com/cassandra/cassandra_architecture.htm



## Add key
You must trust the key below in order to have success.

```
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys A278B781FE4B2BDA
```
## Enable ssh
Generate ssh keys
```
ssh-keygen -t dsa
```
Enable password authentication

In /etc/ssh/sshd_config set
```
PasswordAuthentication yes 
```

Then you can also copy your ssh public key in the VM...

## Download EPC source code


```
ubuntu@test-install-openair-cn:~$ git clone https://github.com/OPENAIRINTERFACE/openair-cn.git
...
ubuntu@test-install-openair-cn:~$ git clone https://github.com/OPENAIRINTERFACE/openair-cn-cups.git
...
```

## Install HSS

**BGN - This part of the wiki is not working due to the java's ppa. Oracle stopped to support it.**

```
ubuntu@test-install-openair-cn:~$ cd openair-cn/scripts/
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ ./build_cassandra --check-installed-software --force
...
Processing triggers for libgdk-pixbuf2.0-0:amd64 (2.36.11-2) ...
Cassandra installation successful
'/home/ubuntu/openair-cn/etc/cassandra.conf' -> '/usr/local/etc/oai/cassandra.conf'
Cassandra has been installed successfully.
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ 
```

~~Set the JRE version 8 as default~~
```
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo service cassandra stop
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo update-alternatives --config java
# select jre version 8
... 
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo service cassandra start
```
Verify that Cassandra is installed and running

**END - This part of the wiki is not working due to the java's ppa. Oracle stopped to support it.**

The message below is shown during the execution of https://github.com/OPENAIRINTERFACE/openair-cn/wiki/OpenAirSoftwareSupport#install-hss, it seems to be missing oracle-java8-installer and there is no valid OpenPGP data.

```
Package oracle-java8-installer is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'oracle-java8-installer' has no installation candidate
deb http://www.apache.org/dist/cassandra/debian 21x main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   227  100   227    0     0    378      0 --:--:-- --:--:-- --:--:--   377
gpg: no valid OpenPGP data found.
```


**See Troubleshoot to fix it**

**Now, we can repeat the steps again**

```
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo service cassandra stop
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo update-alternatives --config java
# select jre version 8
... 
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ sudo service cassandra start
```
Verify that Cassandra is installed and running

```
ubuntu@test-install-openair-cn:~/openair-cn/scripts$ nodetool status
```

The output should look something like:

```
Datacenter: datacenter1
=======================
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address    Load       Tokens  Owns (effective)  Host ID                               Rack
UN  127.0.0.1  51.69 KB   256     100.0%            fa761e10-4118-4a99-9a2e-829e7cdd8e82  rack1
```

Stop Cassandra and cleanup the log files before modifying the configuration

```
sudo service cassandra stop
sudo rm -rf /var/lib/cassandra/data/system/*
sudo rm -rf /var/lib/cassandra/commitlog/*
sudo rm -rf /var/lib/cassandra/data/system_traces/*
sudo rm -rf /var/lib/cassandra/saved_caches/*
```

###  Populate users table
Other problem is shown below
```
Cassandra_Server_IP='127.0.0.1'
cqlsh --file ../src/hss_rel14/db/oai_db.cql $Cassandra_Server_IP
openair-cn@openair-cn-vm:~/openair-cn/scripts$ ./data_provisioning_users --apn default.ng4T.com --apn2 internet --key fec86ba6eb707ed08905757b1bb44b8f --imsi-first 208931234561000 --msisdn-first 001011234561000 --mme-identity mme.ng4T.com --no-of-users 20 --realm ng4T.com --truncate True  --verbose True --cassandra-cluster $Cassandra_Server_IP

Traceback (most recent call last):
  File "./data_provisioning_users", line 178, in <module>
    main()
  File "./data_provisioning_users", line 155, in main
    '{"Subscription-Data":{"Access-Restriction-Data":41,"Subscriber-Status":0,"Network-Access-Mode":2,"Regional-Subscription-Zone-Code":["0x0123","0x4567","0x89AB","0xCDEF","0x1234","0x5678","0x9ABC","0xDEF0","0x2345","0x6789"],"MSISDN":"0x'+str(msisdn)+'","AMBR":{"Max-Requested-Bandwidth-UL":50000000,"Max-Requested-Bandwidth-DL":100000000},"APN-Configuration-Profile":{"Context-Identifier":0,"All-APN-Configurations-Included-Indicator":0,"APN-Configuration":{"Context-Identifier":0,"PDN-Type":0,"Service-Selection":"'+args.apn+'","EPS-Subscribed-QoS-Profile":{"QoS-Class-Identifier":9,"Allocation-Retention-Priority":{"Priority-Level":15,"Pre-emption-Capability":0,"Pre-emption-Vulnerability":0}},"AMBR":{"Max-Requested-Bandwidth-UL":50000000,"Max-Requested-Bandwidth-DL":100000000},"PDN-GW-Allocation-Type":0,"MIP6-Agent-Info":{"MIP-Home-Agent-Address":["172.26.17.183"]}},"APN-Configuration":{"Context-Identifier":0,"PDN-Type":0,"Service-Selection":"'+args.apn2+'","EPS-Subscribed-QoS-Profile":{"QoS-Class-Identifier":9,"Allocation-Retention-Priority":{"Priority-Level":13,"Pre-emption-Capability":1,"Pre-emption-Vulnerability":0}},"AMBR":{"Max-Requested-Bandwidth-UL":50000000,"Max-Requested-Bandwidth-DL":100000000},"PDN-GW-Allocation-Type":0,"MIP6-Agent-Info":{"MIP-Home-Agent-Address":["172.26.17.183"]}}},"Subscribed-Periodic-RAU-TAU-Timer":0}}'
  File "/usr/lib/python3/dist-packages/cassandra/cluster.py", line 1998, in execute
    return self.execute_async(query, parameters, trace, custom_payload, timeout, execution_profile, paging_state).result()
  File "/usr/lib/python3/dist-packages/cassandra/cluster.py", line 3781, in result
    raise self._final_exception
cassandra.cluster.NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 dc1>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level LOCAL_ONE" info={\'consistency\': \'LOCAL_ONE\', \'required_replicas\': 1, \'alive_replicas\': 0}',)})
openair-cn@openair-cn-vm:~/openair-cn/scripts$ 

```
 After change cassandra configuration (**/etc/cassandra/cassandra.yaml**) according to commands below, it works!
```
endpoint_snitch: SimpleSnitch
#endpoint_snitch: GossipingPropertyFileSnitch
```

Erros when execute 
```
penair-cn@openair-cn-vm:~/openair-cn/scripts$ ../src/hss_rel14/bin/make_certs.sh hss ${HSS_CONF[@REALM@]} $PREFIX
Generating a RSA private key
...............................+++++
....................+++++
writing new private key to 'cakey.pem'
-----
Generating RSA private key, 1024 bit long modulus (2 primes)
.......+++++
...................................+++++
e is 65537 (0x010001)
Using configuration from /usr/lib/ssl/openssl.cnf
Can't open ./demoCA/index.txt.attr for reading, No such file or directory
140137160171968:error:02001002:system library:fopen:No such file or directory:../crypto/bio/bss_file.c:72:fopen('./demoCA/index.txt.attr','r')
140137160171968:error:2006D080:BIO routines:BIO_new_file:no such file:../crypto/bio/bss_file.c:79:
Check that the request matches the signature
Signature ok
Certificate Details:
        Serial Number: 1 (0x1)
        Validity
            Not Before: Mar 21 23:42:37 2020 GMT
            Not After : Mar 21 23:42:37 2021 GMT
        Subject:
            countryName               = FR
            stateOrProvinceName       = BdR
            organizationName          = fD
            organizationalUnitName    = Tests
            commonName                = hss.ng4T.com
        X509v3 extensions:
            X509v3 Basic Constraints: 
                CA:FALSE
            Netscape Comment: 
                OpenSSL Generated Certificate
            X509v3 Subject Key Identifier: 
                07:ED:C4:AC:87:75:2A:0F:E4:FC:AC:BA:8F:32:76:FA:EA:2A:A6:8C
            X509v3 Authority Key Identifier: 
                keyid:A9:06:97:71:76:22:76:71:46:01:B6:13:6C:97:49:59:07:17:26:48

Certificate is to be certified until Mar 21 23:42:37 2021 GMT (365 days)

Write out database with 1 new entries
Data Base Updated
'hss.cert.pem' -> '/usr/local/etc/oai/freeDiameter/hss.cert.pem'
'cacert.pem' -> '/usr/local/etc/oai/freeDiameter/cacert.pem'
'hss.key.pem' -> '/usr/local/etc/oai/freeDiameter/hss.key.pem'

```

Error when execute https://github.com/OPENAIRINTERFACE/openair-cn/wiki/OpenAirSoftwareSupport#update-opc-operator-key-ciphered-cassandra-db-if-necessary-do-it-almost-one-time-unless-you-have-only-opc-and-populate-it-in-db

```
openair-cn@openair-cn-vm:~/openair-cn/scripts$ oai_hss -j $PREFIX/hss_rel14.json --onlyloadkey
terminate called after throwing an instance of 'spdlog::spdlog_ex'
  what():  Failed opening file logs/hss.log for writing: No such file or directory
Aborted (core dumped)

```
After google it, I tried to use this solution
https://stackoverflow.com/questions/58843643/understanding-the-error-message-spdlogspdlog-ex
Basically, you create the log file that is missing in /var/log/*.log

### Kill HSS
```
sudo kill -9 $(ps -aux | grep hss | awk '{print $2}')
```

## MME
In _Create MME configuration files_
```
# up S6a
sudo ip addr add 172.16.1.102 brd + dev ens3 label ens3:m11
sudo ip addr add 192.168.247.102 brd + dev ens3 label ens3:m1c
```

## SPGW
In https://github.com/OPENAIRINTERFACE/openair-cn/wiki/OpenAirSoftwareSupport#install-spgw
It is missing fmt library. Run the step below to fix it:

```
cd ~
git clone https://github.com/fmtlib/fmt.git && cd fmt
mkdir _build && cd _build
cmake ..
make -j$(nproc)
sudo make install
```

Install SPGW-U software dependencies
I was crashing qemu after run the command _./build_spgwu -I -f_ using ssh and stopped, I dont know why.

Create SPGW-U configuration files
```
sudo ip addr add 172.55.55.102 brd + dev ens3 label ens3:sxu # SPGW-U SXab interface
sudo ip addr add 192.168.248.159 brd + dev ens3 label ens3:s1u # SPGW-U S1U interface
```

Create SPGW-C configuration files

```
sudo ip addr add 172.55.55.101 brd + dev ens3 label ens3:sxc # SPGW-C SXab interface
sudo ip addr add 172.58.58.102 brd + dev ens3 label ens3:s5c # SGW-C S5S8 interface
sudo ip addr add 172.58.58.101 brd + dev ens3 label ens3:p5c # PGW-C S5S8 interface
sudo ip addr add 172.16.1.104 brd + dev ens3 label ens3:s11 # SGW-C S11 interface

```

## Troubleshoot
**TODO: Check if JAVA installation really needs. The cassandra install open _java-8-openjdk-amd64_ package.**

### How to Install Oracle JAVA JDK8 on Ubuntu
https://github.com/navarrothiago/masters/blob/master/scripts/download_install_jdk.sh

### Check Oracle JAVA JDK8 on Ubuntu
```
java -version
```
The output of the command above should be: 
```
java version "1.8.0_241"
Java(TM) SE Runtime Environment (build 1.8.0_241-b07)
Java HotSpot(TM) 64-Bit Server VM (build 25.241-b07, mixed mode)
```

### Cassandra - no valid OpenPGP data found
Updates https://github.com/OPENAIRINTERFACE/openair-cn/blob/master/build/tools/build_helper.cassandra
```
Replace
curl https://www.apache.org/dist/cassandra/KEYS | $SUDO  apt-key add - 
with
curl https://downloads.apache.org/cassandra/KEYS | $SUDO apt-key add -  
```

## Manage Sections - tmux